# Komoran 형태소 분석기 사용 방법

## 배경
* 형태소 분석기는 general한 말 사용으로 구성
* 비즈니스 과제에서는 내부적 용어 다수 포함  
  --> 형태소 분석기에서 인식 못하는 경우 다수
* 사용자 사전 작성 등을 통한 보완 필요
* 무료 형태소 분석기 중 komoran, kkoma 비교적 높은 평가
* kkoma 에 비해 komoran이 더 빠르고, 품사 태그가 (널리 쓰이는) 세종 말뭉치와 동일

## 목표
* 내부 자료에 맞도록 komoran 형태소분석기 구성 변경 및 사용

## Komoran 소개

* Original Komoran in Java
    * [komoran v3](https://github.com/shin285/KOMORAN), 비교적 최근 업데이트
    * [komoran v2](https://github.com/shineware/komoran-2.0)
* Komoran in Python  
    * 자바 버전 내부적으로 호출
    * [PyKomoran](https://github.com/shineware/PyKOMORAN)
    * [komoran3py](https://github.com/lovit/komoran3py)
    * [Konlpy](https://konlpy-ko.readthedocs.io/ko/v0.5.1/api/konlpy.tag/#module-konlpy.tag._komoran)
        * 최신 버전은 0.5.1
        * version 0.5 이후 Komoran v3 적용  
        * Komoran API 는 Komoran class 내 세 개의 method 로 구성
        
```java
# Komoran class in Konlpy
class konlpy.tag._komoran.Komoran(jvmpath=None, userdic=None, 
                                  modelpath=None, max_heap_size=1024){
    morphs(phrase) : Parse phrase to morphemes.
    nouns(phrase) : Noun extractor.
    pos(phrase, flatten=True, join=False) : POS tagger.
}
```

## Komoran 사용법
* 기본 모델 사용하기
    * 예) Konlpy 사용
    * 단점 :
        * 사용자 단어 사용시 속도 저하
        * 많은 단어 추가 시 작동 원활하지 않음
            * ~ 15만 개 추가시 작동 오류 및 메모리 부족 발생
        * 자체 사전 크기 적음
            * default ~3만 형태소 사용  
            * kkoma(~30만), 세종 코퍼스(~20만)
* 별도 모델 개발 후 사용하기
    * 다수의 사용자 단어 추가 가능
    * Komoran 모델 재구성 --> Komoran instance 구성에서 신규 모델 로드  
    
```java
class konlpy.tag._komoran.Komoran(jvmpath=None, userdic=None, 
                                  <span style="color:red">modelpath=None</span>, 
                                  max_heap_size=1024)
```

> 단어 숫자는 얼마나 필요한가?  
* 최근 딥러닝 모델들은 수만 개 정도의 단어 사용  
* 위에서 언급한 단어 수는 형태소 분석에 사용  
* 실제 딥러닝에서 사용되는 단어 수와는 다를 수 있음

## Komoran custom 구성 방법
* [참조] [komoran 공식 문서](https://docs.komoran.kr/?utm_source=komoran-repo&utm_medium=Referral&utm_campaign=github-demo) : 설명 자세하지 않아 API 부분 확인 필요  
* konlpy 내 다른 형태소 분석기와는 다르게 komoran 사전은 모델 내 간접적으로 포함
* default : stable 모델, github 내 full 모델 제공 --> 사용자 모델 필요

### komoran 수정 방법
 
1. corpus build 
    * 관심 텍스트 문서를 기반으로 사전 파일 생성  
      예) github 내 corpus_build 디렉토리  
2. model build
    * 사전 파일을 기반으로 모델 파일 생성  
      예) github 내 models_full, models_light 디렉토리  
3. model load
    * 모델 파일 위치 설정

## Komoran 모델 build 예시
아래 방식을 참고하여 사용하고자 하는 데이터 별로 적용 가능  

### 1. corpus_build 내 기본 사전(dic.word) 이용
* 텍스트 파일로 구성
* 각 형태소와 형태소의 빈도 기록
* 한 개 형태소는 여러 품사 가질 수 있음
>```꼬지	NNG:5  
탄로	NNG:10```

## Komoran 모델 생성 예시 (계속)
### 2. 세종말뭉치 파일 전처리
#### [세종 말뭉치](https://ithub.korean.go.kr/user/main.do)
* 국립국어원에서 배포한 자료로 한글 분석에서 널리 사용되는 기본 자료  
* 국립국어원 언어정보나눔터 --> 말뭉치 --> 기타 참고자료 --> 본인 필요자료 이용
* 현대구어 형태분석 말뭉치, 현대문어 형태분석 말뭉치 자료 사용  
  예시) [현대구어 형태 분석](5CT_0013.txt), [현대구어 형태 분석](BTAA0001.txt)
* 원본 파일에 대한 전처리 필요 --> 아래 공개 소스 활용  

#### [sejong_corpus_cleaner](https://github.com/lovit/sejong_corpus_cleaner)
* 세종말뭉치는 2차 배포가 자유롭지 않아 직접 파일을 받아 전처리 스크립트 실행 필요
* 설명을 따라 디렉토리, 파일 구축 후 script 실행
    * scripts/build_eojeol_morphtag.py, scripts/build_eojeol_morphtag_table.py

> sejong_corpus_cleaner/scripts/build_eojeol_morphtag_table.py
    * inputs : 세종말뭉치 원본 파일
    * outputs : eojeol_morphtag_table_written.txt,  
                eojeol_morphtag_table_colloquial.txt  

## Komoran 모델 생성 예시 (계속)
### 3. 세종말뭉치 단어 기본 사전에 추가
* 세종 말뭉치의 경우 사람이 작업한 라벨이므로 추가

#### [get_sejong_corpus.ipynb](get_sejong_corpus.ipynb)

* 전처리 파일로부터 세종 말뭉치 내 형태소 추출 후 sejong.dic 으로 저장
* inputs : eojeol_morphtag_table_written.txt, eojeol_morphtag_table_colloquial.txt
* outputs : sejong.dic

#### [build_word_dic_v2.ipynb](build_word_dic_v2.ipynb)

* 기존 사전과 세종 말뭉치 합하여 dic2.word 로 저장
* inputs : sejong.dic, dic.word
* outputs : dic2.word          
* sejong_corpus_cleaner 에서 계산된 발생 빈도 합산해 업데이트

> 전처리 유의 사항 (예외 case)  
* 기호 등이 있는 경우 "" 로 형태소 묶어 기록  
  일반 case) 달려가 --> 달려가, 예외 case) , 달려가 --> ", 달려가 "  
* 세종 말뭉치 내 일부 tag 은 komoran tag 에 없음  
  (구어에서만 발생, komoran 이 Konlpy 와 동일한 tag 을 쓴다고 알려져 있으나  
   세종말뭉치에 예외적인 tag들 있음)  

##  Komoran 모델 생성 예시 (계속)

### 4. 신규 단어 추출

#### [noun_extractor_v2.ipynb](noun_extractor_v2.ipynb) : 
* 신규 단어도 기본 단어장에 포함시키는 것이 유리
* soynlp 명사 추출기 이용하여 신규 단어 추출 후 기존 사전에 추가
* inputs : dic2.word
* outputs : dic2.word (rename later with dic.word)  

> 적용 구성 
* noun_extractor v1, v2 중 v1 사용  
* 2018년 1월부터 10월 까지 뉴스 ~40만건 사용
* ~40000 형태소 신규 추출 후 기존 사전에 없는 ~15000 개 선택  
* 명사추출기(통계적 추출)는 사용 문서 양이 많을수록 정확도 향상   
   --> 50000만 뉴스씩 뭉치 구성    
* 명사 확률값이 나오므로 확률 값, 빈도 등에 대한 필터링 가능  
(빈도\*확률 값을 신규 빈도로 적용)
* 추출 명사에 대해 한글만 추가  

## Komoran 모델 생성 예시 (계속)
### 5. 신규 모델 생성
#### [ModelBuildTest.java](ModelBuildTest.java)
* inputs : dic.word (in corpus_build)
* outputs : observation.model (in models)
```java
ModelBuilder builder = new ModelBuilder();
builder.setExternalDic("../corpus_build/user.dic");
builder.buildPath("../corpus_build");
builder.save("models");
```
* 실행 
    1. 신규 dic.word 를 사전 폴더 (default : corpus_build) 에 넣기   
    2. Komoran.jar 생성 :  
        * docs.komoran.kr 참조  
          ( --> 설치하기 --> Gradle 이용하기, Jar 파일 만들기)
    3. 컴파일
        * javac -cp ".:./libs/KOMORAN.jar:" ModelBuildTest.java 
    4. 모델 생성
        * java -cp ".:./libs/KOMORAN.jar:" ModelBuildTest
    5. 모델 로드
        * komoran instance 호출시 모델 경로 설정

            ```python
            # PyKomoran 이용할 경우,
            import PyKomoran as komo
            komoran = komo.Komoran(".../build/models")
            anal4 = komoran4.get_plain_text(content1)            
            ```  

## Komoran 모델 생성 예시 (계속)

### 6. 복합 형태소 축약(optional)
* 적용 전후 결과 비교 등을 통해 적용 여부 판단 필요

#### [morpheme_analyzer_v2.ipynb](morpheme_analyzer_v2.ipynb)
* 새롭게 만든 모델로 형태소분석기 호출
* 분석할 문장에 형태소 분석기 적용
* L-R corpus 로 변환 (https://github.com/lovit/sejong_corpus_cleaner)

## 형태소 분석 변화

> __stable model 사용 시__  
            신정/NNG 3/SN 동/NNB 신정/NNG 뉴/NNG 타운/NNG 2/SN -/SW 1/SN 구역/NNG 을/JKO  
            재/XPN 개발/NNG 하/XSV 는/ETM 이/MM 단지/MAG 는/JX 2/SN 호선/NNG 지선/NNG 이/VCP   
            ㄴ/ETM 신정/NNG 네거리/NNG 역/NNG 의/JKG 초/XPN 역세권/NNG 에/JKB 위치/NNG 하/XSV ㄴ다/EC 
>            
> __신규 모델 사용 시__  
            신정/NNG 3/SN 동/NNB 신정뉴타운/NNP 2/SN -/SW 1/SN 구역/NNG 을/JKO 재개발/NNG 하/XSV  
            는/ETM 이/MM 단지/NNG 는/JX 2/SN 호선/NNB 지선/NNG 이/VCP ㄴ/ETM 신정네거리역/NNP 의/JKG  
            초역세권/NNP 에/JKB 위치/NNG 하/XSV ㄴ다/EC
>              
> __LR corpus 적용 후__              